# imports

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PreTrainedModel, LlamaTokenizer, LlamaForCausalLM, pipeline
from dotenv import load_dotenv
import os
from datasets import load_from_disk
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, PeftModel
import torch

c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
#load hugging-face token
load_dotenv()
hf_token = os.environ["HF_ACCESS_TOKEN"]

#set wandb environment variables
os.environ["WANDB_ENTITY"] = "t_buess"
os.environ["WANDB_PROJECT"] = "chatbot-qa"

# parameters

In [3]:
base_model_id = "meta-llama/Llama-2-7b-hf"
finetuned_path = "./data/models/llama2-fine-tuned"
ft_dataset_filename = "data/processed/ft_dataset.hf"
train_set_text_field = "text"
batch_size = 1
grad_accumulation_steps = 4
optimizer = "paged_adamw_32bit"
learning_rate = 1e-4
max_steps = 100
max_seq_length = 4096
device_map="auto"

# configuration

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# load dataset

In [ ]:
def formatter(example):
    prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext sowie der passenden Antwort"
        "Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein"
        "### Frage:\n"
        f"{example['question']}\n\n"
        "### Kontext:\n"
        f"{example['context']}\n\n"
        "### Antwort:\n"
        f"{example['answers']}"
    )

    return {"text": prompt}

#load train dataset
train_dataset = load_from_disk(ft_dataset_filename)

#add text column
train_dataset = train_dataset.map(formatter)

: 

# load tokenizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    token=hf_token,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    token=hf_token,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

: 

# train

In [ ]:
train_args = TrainingArguments(
    output_dir=finetuned_path + "/train-out",
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accumulation_steps,
    learning_rate=learning_rate,
    optim=optimizer,
    logging_steps=1,
    max_steps=max_steps,
    lr_scheduler_type="constant",
    gradient_checkpointing=True
)

# LoRA adapters added inside the trainer, prepare_model_for_kbit_training is called here
fine_tuning = SFTTrainer(
    model=base_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field=train_set_text_field,
    max_seq_length=max_seq_length,
    args=train_args
)

fine_tuning.train()

: 

# save model and tokenizer

In [ ]:
fine_tuning.model.save_pretrained(finetuned_path + "/model")
fine_tuning.tokenizer.save_pretrained(finetuned_path + "/tokenizer")

: 

# inference

In [ ]:
def predict(model:PreTrainedModel, tokenizer:LlamaTokenizer, question:str, context:str):
    model.eval()
    
    prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext\n"
        "Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein\n\n"
        "### Frage:\n"
        f"{question}\n\n"
        "### Kontext:\n"
        f"{context}\n\n"
        "### Antwort:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda:0"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.pad_token_id, do_sample=True)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

question = "Wer hat python erfunden?"
context = (
    "Die Sprache wurde Anfang der 1990er Jahre von Guido van Rossum am Centrum Wiskunde & Informatica in Amsterdam als Nachfolger für die Programmier-Lehrsprache ABC entwickelt" 
    "und war ursprünglich für das verteilte Betriebssystem Amoeba gedacht."
    "Der Name geht nicht, wie das Logo vermuten lässt, auf die gleichnamige Schlangengattung Python zurück, sondern bezog sich ursprünglich auf die englische Komikergruppe Monty Python." 
    "In der Dokumentation finden sich daher auch einige Anspielungen auf Sketche aus dem Flying Circus. Trotzdem etablierte sich die Assoziation zur Schlange, was sich unter anderem" 
    "in der Programmiersprache Cobra[16] sowie dem Python-Toolkit „Boa“[17] äußert. Die erste Vollversion erschien im Januar 1994 unter der Bezeichnung Python 1.0." 
    "Gegenüber früheren Versionen wurden einige Konzepte der funktionalen Programmierung implementiert, die allerdings später wieder aufgegeben wurden.[18] Von 1995 bis 2000 erschienen neue Versionen,"
    "die fortlaufend als Python 1.1, 1.2 etc. bezeichnet wurden."
)
    

predict(base_model, tokenizer, question, context)

: 

# load model

In [6]:
#TODO